In [ ]:
%pip install sqlite_vec

from dotenv import load_dotenv
from openai import OpenAI
load_dotenv()  # take environment variables from .env.

In [ ]:
import os
import sys
import json
validation_set = []

for folder in os.listdir('./'):
    if not os.path.isdir(folder):
        continue
    for file in os.listdir(f'./{folder}'):
        if not file.endswith('.json'):
            continue
        data = open(f'./{folder}/'+file, 'r', encoding="utf-8").read()
        data = json.loads(data)
        
        relevant_messages = []
        for index, mesg in enumerate(data['messages']):
            print(mesg.get('content', ''))
            if mesg['role'] == 'system':
                relevant_messages.append(mesg)
            elif mesg.get('content', '') and mesg['content'].startswith('('):
                # Always keep context messages (status / events)
                relevant_messages.append(mesg)
            elif index > len(data['messages'])-18:
                # Only keep the last 18 messages, as anything before 20 that is truncated and only kept for context, and never produced by an actual request
                relevant_messages.append(mesg)
        
        messages = []
        for mesg in relevant_messages:
            messages.append({})
            messages[-1]['role'] = mesg['role']
            if 'content' in mesg and mesg['content']:
                messages[-1]['content'] = mesg['content']
            else:
                messages[-1]['content'] = ''
            if 'tool_calls' in mesg and mesg['tool_calls']:
                messages[-1]['tool_calls'] = mesg['tool_calls']
        
        tools = data['tools']
        
        validation_set.append({
            'messages': json.dumps(messages),
            'tools': json.dumps(tools),
        })

print(f'Loaded {len(validation_set)} validation sets')
with open('validation_set.jsonl', 'w', encoding="utf-8") as f:
    for item in validation_set:
        f.write(json.dumps(item)+'\n')
        